In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf


from tensorflow.keras import datasets, layers, models

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# tf.config.gpu.set_per_process_memory_growth(True)
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
# tf.config.gpu.set_per_process_memory_fraction(0.4)

In [ ]:
path = "mnist.npz"
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data(path)

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images[:8192]
test_labels = test_labels[:8192]

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))


    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=5,batch_size=40960)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, batch_size=4096)

In [ ]:
print(test_acc)